### Imports

In [ ]:
import os
import pyspark
import json
from pyspark.sql import SparkSession

In [ ]:
# Get metadata for tables to check | Location can be parameterised

metadata_path = '..\\output_obj\\tbl_metadata.json'

with open(metadata_path, "r") as f:
    metadata = json.load(f)

print("Loaded JSON data:", metadata)

### Setup spark session

In [ ]:
spark = SparkSession.builder \
    .appName("MyDockerSparkApp") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.0,com.microsoft.azure:azure-storage:8.6.6") \
    .getOrCreate()

# Debug via docker
spark.conf.set("fs.azure.impl", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
print("PySpark version:", pyspark.__version__)
hadoop_version = spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()
print("Hadoop version:", hadoop_version)  

### Create normalised tables

In [ ]:
# Catalog and schema information | Can be parameterised in actual usecase
catalog_name = 'data_foundation_dev'
src_schema_name = 'raw'
tgt_schema_name = 'stg'

In [ ]:
# Iterate through sql folder and run cts

sql_path = '..\\sql_files_normalised\\'

for file in os.listdir(sql_path):
    if file.endswith(".sql"):
        file_path = sql_path+file
        with open(file_path, "r") as f:
            ctas = f.read()
            run = spark.sql(ctas)
            print(f"Tbl created: {file}")